In [1]:
import yfinance as yf 
import numpy as np 
import QuantLib as ql

In [2]:
ticker = 'KO'
data = yf.download(ticker, period='10y', interval='1d', auto_adjust=True, progress=False)
data['returns'] = np.log(data.Close / data.Close.shift(1))
data.dropna(inplace=True)
data.tail()

,Open,High,Low,Close,Volume,returns
Date,,,,,,
2022-02-17,60.549999,62.279999,60.410000,62.119999,21468700,0.019835
2022-02-18,62.119999,62.820000,61.990002,62.540001,21968400,0.006738
2022-02-22,62.430000,62.639999,61.720001,62.279999,21731400,-0.004166
2022-02-23,62.240002,62.330002,61.340000,61.590000,19248500,-0.011141
2022-02-24,59.970001,60.619999,59.220001,60.509998,27149054,-0.017691


In [3]:
S_0 = data.Close[-1]
sigma = data.returns.std() * np.sqrt(252)
N = 252 # 252 trading days
K = 60 # strike price
rf = 0.01 # risk-free rate
num_sim = 100_000

In [4]:
calendar = ql.UnitedStates()
day_counter = ql.ActualActual()

In [5]:
valuation_date = ql.Date(24, 2, 2022)
expiry_date = ql.Date(24, 2, 2023)
ql.Settings.instance().evaluationDate = valuation_date

In [6]:
call_type = ql.Option.Call
put_type = ql.Option.Put

exercise = ql.AmericanExercise(valuation_date, expiry_date)
call_payoff = ql.PlainVanillaPayoff(call_type, K)
put_payoff = ql.PlainVanillaPayoff(put_type, K)

In [7]:
u = ql.SimpleQuote(S_0)
r = ql.SimpleQuote(rf)
sigma = ql.SimpleQuote(sigma)

In [8]:
underlying = ql.QuoteHandle(u)
volatility = ql.BlackConstantVol(0, ql.TARGET(), ql.QuoteHandle(sigma), day_counter)
risk_free_rate = ql.FlatForward(0, ql.TARGET(), ql.QuoteHandle(r), day_counter)

In [9]:
bs_process = ql.BlackScholesProcess(underlying, 
                                    ql.YieldTermStructureHandle(risk_free_rate),
                                    ql.BlackVolTermStructureHandle(volatility))

In [10]:
engine = ql.MCAmericanEngine(bs_process, 'PseudoRandom', timeSteps=N, polynomOrder=5, 
                             seedCalibration=42, requiredSamples=num_sim)

In [11]:
call_option = ql.VanillaOption(call_payoff, exercise)
call_option.setPricingEngine(engine)
call_premium = call_option.NPV()

put_option = ql.VanillaOption(put_payoff, exercise)
put_option.setPricingEngine(engine)
put_premium = put_option.NPV()

In [12]:
print(f"Strike Price: {K}")
print(f"Call premium: {call_premium:.4f}")
print(f"Put premium: {put_premium:.4f}")

Strike Price: 60
Call premium: 4.6784
Put premium: 3.6507
